In [15]:
# For tips on running notebooks in Google Colab, see
# https://pytorch.org/tutorials/beginner/colab
%matplotlib inline

[Learn the Basics](intro.html) \|\|
[Quickstart](quickstart_tutorial.html) \|\|
[Tensors](tensorqs_tutorial.html) \|\| [Datasets &
DataLoaders](data_tutorial.html) \|\|
[Transforms](transforms_tutorial.html) \|\| **Build Model** \|\|
[Autograd](autogradqs_tutorial.html) \|\|
[Optimization](optimization_tutorial.html) \|\| [Save & Load
Model](saveloadrun_tutorial.html)

Build the Neural Network
========================

Neural networks comprise of layers/modules that perform operations on
data. The [torch.nn](https://pytorch.org/docs/stable/nn.html) namespace
provides all the building blocks you need to build your own neural
network. Every module in PyTorch subclasses the
[nn.Module](https://pytorch.org/docs/stable/generated/torch.nn.Module.html).
A neural network is a module itself that consists of other modules
(layers). This nested structure allows for building and managing complex
architectures easily.

In the following sections, we\'ll build a neural network to classify
images in the FashionMNIST dataset.


In [16]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

Get Device for Training
=======================

We want to be able to train our model on a hardware accelerator like the
GPU or MPS, if available. Let\'s check to see if
[torch.cuda](https://pytorch.org/docs/stable/notes/cuda.html) or
[torch.backends.mps](https://pytorch.org/docs/stable/notes/mps.html) are
available, otherwise we use the CPU.


In [17]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


Define the Class
================

We define our neural network by subclassing `nn.Module`, and initialize
the neural network layers in `__init__`. Every `nn.Module` subclass
implements the operations on input data in the `forward` method.


In [18]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

We create an instance of `NeuralNetwork`, and move it to the `device`,
and print its structure.


In [19]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


To use the model, we pass it the input data. This executes the model\'s
`forward`, along with some [background
operations](https://github.com/pytorch/pytorch/blob/270111b7b611d174967ed204776985cefca9c144/torch/nn/modules/module.py#L866).
Do not call `model.forward()` directly!

Calling the model on the input returns a 2-dimensional tensor with dim=0
corresponding to each output of 10 raw predicted values for each class,
and dim=1 corresponding to the individual values of each output. We get
the prediction probabilities by passing it through an instance of the
`nn.Softmax` module.


In [20]:
X = torch.rand(1, 28, 28, device=device)
print(X)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

tensor([[[3.5568e-01, 2.7180e-01, 5.6373e-01, 2.4500e-01, 9.1931e-01,
          5.6184e-01, 8.1010e-01, 9.3863e-01, 6.2271e-02, 6.7647e-01,
          1.9217e-01, 9.3962e-01, 9.0039e-01, 2.2040e-01, 3.3428e-01,
          5.0134e-01, 3.9216e-01, 3.0749e-02, 1.1921e-01, 7.2443e-01,
          8.2987e-01, 6.2187e-01, 5.6035e-01, 9.8203e-01, 3.1890e-01,
          3.1358e-03, 9.5026e-01, 8.1826e-03],
         [5.6772e-03, 4.0883e-01, 6.4094e-01, 7.4965e-01, 3.9148e-01,
          4.9656e-01, 2.1115e-01, 3.7513e-01, 8.5457e-01, 7.5076e-01,
          9.6209e-01, 4.1814e-01, 5.0328e-01, 2.9652e-01, 9.7557e-01,
          2.7621e-01, 2.2855e-01, 8.1416e-01, 6.3689e-01, 2.1736e-01,
          1.7673e-01, 4.9862e-04, 6.4536e-01, 2.5573e-01, 4.9815e-01,
          5.8634e-01, 9.2611e-01, 3.2190e-01],
         [9.8041e-01, 3.8452e-01, 7.7122e-01, 2.9518e-01, 1.2140e-01,
          2.0616e-01, 5.8197e-01, 2.1342e-02, 5.1038e-01, 9.8341e-01,
          3.5087e-01, 8.3930e-03, 5.8868e-01, 4.9975e-01, 1.8977e-

------------------------------------------------------------------------


Model Layers
============

Let\'s break down the layers in the FashionMNIST model. To illustrate
it, we will take a sample minibatch of 3 images of size 28x28 and see
what happens to it as we pass it through the network.


In [21]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


nn.Flatten
==========

We initialize the
[nn.Flatten](https://pytorch.org/docs/stable/generated/torch.nn.Flatten.html)
layer to convert each 2D 28x28 image into a contiguous array of 784
pixel values ( the minibatch dimension (at dim=0) is maintained).


In [22]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


nn.Linear
=========

The [linear
layer](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html)
is a module that applies a linear transformation on the input using its
stored weights and biases.


In [23]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


nn.ReLU
=======

Non-linear activations are what create the complex mappings between the
model\'s inputs and outputs. They are applied after linear
transformations to introduce *nonlinearity*, helping neural networks
learn a wide variety of phenomena.

In this model, we use
[nn.ReLU](https://pytorch.org/docs/stable/generated/torch.nn.ReLU.html)
between our linear layers, but there\'s other activations to introduce
non-linearity in your model.


In [24]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[-0.4565, -0.1312,  0.3884, -0.2676,  0.2631,  0.1213,  0.5622, -0.3696,
         -0.2753, -0.0062, -0.2187, -0.0516,  0.0022,  0.1075,  0.3042, -0.2665,
         -0.2184, -0.1354, -0.0346,  0.9301],
        [-0.1222, -0.0902,  0.0412, -0.4622,  0.1229, -0.2050,  0.7770, -0.2813,
         -0.1728,  0.4880,  0.0491, -0.3908,  0.1625, -0.2729,  0.2139, -0.0023,
          0.0456,  0.2518,  0.1366,  0.3197],
        [-0.1233, -0.4204,  0.6116,  0.2102,  0.1018,  0.0362,  0.4355, -0.0485,
         -0.6380, -0.0269, -0.0313, -0.2464, -0.1325, -0.0880,  0.2883, -0.2398,
          0.0397,  0.1373, -0.0653,  0.6727]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0000, 0.0000, 0.3884, 0.0000, 0.2631, 0.1213, 0.5622, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0022, 0.1075, 0.3042, 0.0000, 0.0000, 0.0000,
         0.0000, 0.9301],
        [0.0000, 0.0000, 0.0412, 0.0000, 0.1229, 0.0000, 0.7770, 0.0000, 0.0000,
         0.4880, 0.0491, 0.0000, 0.1625, 0.0000, 0.21

nn.Sequential
=============

[nn.Sequential](https://pytorch.org/docs/stable/generated/torch.nn.Sequential.html)
is an ordered container of modules. The data is passed through all the
modules in the same order as defined. You can use sequential containers
to put together a quick network like `seq_modules`.


In [25]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

nn.Softmax
==========

The last linear layer of the neural network returns [logits]{.title-ref}
- raw values in \[-infty, infty\] - which are passed to the
[nn.Softmax](https://pytorch.org/docs/stable/generated/torch.nn.Softmax.html)
module. The logits are scaled to values \[0, 1\] representing the
model\'s predicted probabilities for each class. `dim` parameter
indicates the dimension along which the values must sum to 1.


In [26]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

Model Parameters
================

Many layers inside a neural network are *parameterized*, i.e. have
associated weights and biases that are optimized during training.
Subclassing `nn.Module` automatically tracks all fields defined inside
your model object, and makes all parameters accessible using your
model\'s `parameters()` or `named_parameters()` methods.

In this example, we iterate over each parameter, and print its size and
a preview of its values.


In [27]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[-0.0275,  0.0120, -0.0097,  ..., -0.0185, -0.0072,  0.0146],
        [ 0.0010, -0.0347,  0.0064,  ...,  0.0318,  0.0217, -0.0027]],
       device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([-0.0102,  0.0098], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[ 0.0014, -0.0153,  0.0316,  ..., -0.0332,  0.0415, -0.0244],
        [-0.0400, -0.0265,  0.0255,  ...,  0.0240,  0.0110, -0.0284]],
       device='cuda:0', grad_fn=<Sl

------------------------------------------------------------------------


Further Reading
===============

-   [torch.nn API](https://pytorch.org/docs/stable/nn.html)
